<a href="https://colab.research.google.com/github/MuhammadTaha25/Deep-Learning/blob/main/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the Fashion MNIST dataset (70,000 grayscale images of clothing items)
from tensorflow.keras.datasets import fashion_mnist

# Import the Sequential and Model classes to define neural network architectures
from tensorflow.keras.models import Sequential, Model

# Import layers:
#  - Input: to define the input tensor
#  - Reshape: to reshape tensors
#  - Dense: for fully connected layers
#  - Conv2D: for 2D convolutional layers
#  - MaxPooling2D: for downsampling feature maps
#  - UpSampling2D: for upsampling in decoder networks
#  - Flatten: to flatten feature maps into vectors
from tensorflow.keras.layers import Input, Reshape, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten

# Import utility to convert integer labels to one-hot encoded vectors
from tensorflow.keras.utils import to_categorical

# Import Cropping2D layer in case you need to crop input images
from tensorflow.keras.layers import Cropping2D

# Matplotlib for plotting images and training curves
import matplotlib.pyplot as plt

# NumPy for numerical operations on arrays
import numpy as np


In [ ]:
# Load the Fashion MNIST dataset:
#   • (X, Y) are the training images (X) and their labels (Y)
#   • (x, y) are the test images (x) and their labels (y)
(X, Y), (x, y) = fashion_mnist.load_data()


In [ ]:
# Reshape training images to 4D tensors:
#   • X.shape[0]: number of training samples
#   • 28, 28: image width and height
#   • 1: single channel for grayscale
X = X.reshape(X.shape[0], 28, 28, 1)

# Reshape test images similarly to include the channel dimension
x = x.reshape(x.shape[0], 28, 28, 1)


In [ ]:
# Convert pixel values from integers to 32-bit floats for numerical stability
x = x.astype('float32')
X = X.astype('float32')

# Normalize pixel values to the range [0, 1] by dividing by the maximum value 255
x /= 255.0
X /= 255.0


In [ ]:
# Convert integer class labels to one-hot encoded vectors for 10 classes
y = to_categorical(y, 10)  # Test labels
Y = to_categorical(Y, 10)  # Training labels


In [ ]:
type(X)

numpy.ndarray

In [ ]:
# Define the input tensor for the model:
#   • shape=(28, 28, 1) corresponds to 28×28 pixels with 1 channel (grayscale)
input_img = Input(shape=(28, 28, 1))


In [ ]:
# Define a new input layer for the convolutional autoencoder
# Shape: 28×28 pixels, 1 channel (grayscale)
input_layer_cnv = Input(shape=(28, 28, 1))

# ---------------------
# Encoder
# ---------------------

# First convolution + pooling block
# • 32 filters of size 3×3, ReLU activation
# • ‘same’ padding preserves spatial dimensions
ae_cnv_en = Conv2D(32, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(input_layer_cnv)
ae_cnv_en = MaxPooling2D((2, 2), padding="same")(ae_cnv_en)

# Second convolution + pooling block
# • 32 filters of size 3×3, ReLU activation
# • ‘same’ padding to keep feature map size
ae_cnv_en = Conv2D(32, (3, 3), activation="relu", padding="same")(ae_cnv_en)
ae_cnv_en = MaxPooling2D((2, 2), padding="same")(ae_cnv_en)

# Third convolution + pooling block
# • 4 filters of size 3×3, ReLU activation
# • This drastically reduces channels for a compact bottleneck
ae_cnv_en = Conv2D(4, (3, 3), activation="relu", padding="same")(ae_cnv_en)
ae_cnv_en = MaxPooling2D((2, 2), padding="same")(ae_cnv_en)

# Flatten the feature maps into a vector (the “bottleneck” representation)
ae_cnv_en = Flatten(name="bot")(ae_cnv_en)

# ---------------------
# Decoder
# ---------------------

# Reshape the bottleneck vector back into a small feature map
# Input shape for Reshape must match the flattened size: here 4×4×4 = 64
ae_cnv_de = Reshape((4, 4, 4), input_shape=(64,), name="botnext0")(ae_cnv_en)

# First upsampling + convolution block
# • Upsample by a factor of 2 (4×4 → 8×8)
# • 4 filters of size 3×3, ReLU activation
ae_cnv_de = Conv2D(4, (3, 3), activation="relu", padding="same", name="botnext1")(ae_cnv_de)
ae_cnv_de = UpSampling2D((2, 2), name="botnext2")(ae_cnv_de)

# Second upsampling + convolution block
# • Upsample by 2 again (8×8 → 16×16)
# • 32 filters of size 3×3, ReLU activation
ae_cnv_de = Conv2D(32, (3, 3), activation="relu", padding="same", name="botnext3")(ae_cnv_de)
ae_cnv_de = UpSampling2D((2, 2), name="botnext4")(ae_cnv_de)

# Third upsampling + convolution block
# • Upsample by 2 (16×16 → 32×32)
# • 32 filters, but note padding="valid" here will reduce spatial dims slightly
ae_cnv_de = Conv2D(32, (3, 3), activation="relu", padding="valid", name="botnext5")(ae_cnv_de)
ae_cnv_de = UpSampling2D((2, 2), name="botnext6")(ae_cnv_de)

# Final convolution to map back to single-channel image
# • 1 filter of size 3×3, sigmoid activation to produce outputs in [0,1]
ae_cnv_de = Conv2D(1, (3, 3), activation="sigmoid", padding="same", name="botnext7")(ae_cnv_de)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "functional_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_63 (Conv2D)              │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_43 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_64 (Conv2D)              │ (None, 14, 14, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_44 (MaxPooling2D) │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_65 (Conv2D)              │ (None, 7, 7, 4)        │         1,156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_45 (MaxPooling2D) │ (None, 4, 4, 4)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bot (Flatten)                   │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ botnext0 (Reshape)              │ (None, 4, 4, 4)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ botnext1 (Conv2D)               │ (None, 4, 4, 4)        │           148 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ botnext2 (UpSampling2D)         │ (None, 8, 8, 4)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ botnext3 (Conv2D)               │ (None, 8, 8, 32)       │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ botnext4 (UpSampling2D)         │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ botnext5 (Conv2D)               │ (None, 14, 14, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ botnext6 (UpSampling2D)         │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ botnext7 (Conv2D)               │ (None, 28, 28, 1)      │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,593 (84.35 KB)

 Trainable params: 21,593 (84.35 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Instantiate the autoencoder model, specifying inputs and outputs
Ae_Conv = Model(inputs=input_layer_cnv, outputs=ae_cnv_de)

# Compile the model with:
# • Adam optimizer
# • Binary crossentropy loss for pixel-wise comparison
# • Track accuracy metric (though visual quality is often a better judge)
Ae_Conv.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

# Alternative optimizer example (commented out):
# Ae_Conv.compile(optimizer=tf.keras.optimizers.Adadelta(0.1, clipvalue=2),
#                 loss='binary_crossentropy', metrics=["accuracy"])

# Print the model architecture summary
Ae_Conv.summary()

In [ ]:
# Create a sub-model that starts at the decoder’s entry point (just before the first reshape)
# and outputs the tensor right after that reshape layer.
# This lets you inspect or use the decoder’s input feature maps directly.
ae_cnv_en = Model(
    inputs=Ae_Conv.get_layer('bottleneck_reshape').input,  # the input to the Reshape layer
    outputs=Ae_Conv.get_layer('botnext0').output           # the output of the Reshape layer
)


In [ ]:
# Encoder model summary
ae_cnv_en.summary()

Model: "functional_41"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_63 (Conv2D)  │ (None, 28, 28,    │        320 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_43    │ (None, 14, 14,    │          0 │ conv2d_63[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_64 (Conv2D)  │ (None, 14, 14,    │      9,248 │ max_pooling2d_43… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_44    │ (None, 7, 7, 32)  │          0 │ conv2d_64[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_65 (Conv2D)  │ (None, 7, 7, 4)   │      1,156 │ max_pooling2d_44… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_45    │ (None, 4, 4, 4)   │          0 │ conv2d_65[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bot (Flatten)       │ (None, 64)        │          0 │ max_pooling2d_45… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,724 (41.89 KB)

 Trainable params: 10,724 (41.89 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Create a decoder sub-model that maps from the bottleneck’s reshaped feature maps
# directly through to the final decoder layer named "decode8".
# This lets you run just the decoder portion, starting at its Reshape input,
# and inspect or generate output halfway through decoding.
ae_cnv_de = Model(
    inputs=autoencoder.get_layer('bottleneck_reshape').input,  # 4×4×4 feature map entering the decoder
    outputs=autoencoder.get_layer('decode8').output           # output of the layer "decode8"
)


In [ ]:
# Encoder model summary
ae_cnv_de.summary()

Model: "functional_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ keras_tensor_298CLONE           │ (None, 36)             │             0 │
│ (InputLayer)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bottleneck_reshape (Reshape)    │ (None, 3, 3, 4)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode2 (Conv2D)                │ (None, 3, 3, 4)        │           148 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode3 (UpSampling2D)          │ (None, 6, 6, 4)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode4 (Conv2D)                │ (None, 6, 6, 32)       │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode5 (UpSampling2D)          │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode6 (Conv2D)                │ (None, 12, 12, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode7 (UpSampling2D)          │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode8 (Conv2D)                │ (None, 24, 24, 1)      │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,869 (42.46 KB)

 Trainable params: 10,869 (42.46 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the decoder-input sub-model:
#  • loss='categorical_crossentropy' – measures the difference between predicted and true distributions
#  • optimizer='adam' – adaptive gradient-based optimizer
#  • metrics=['accuracy'] – track accuracy during training/evaluation
ae_cnv_en.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [ ]:
# Train the convolutional autoencoder:
#  • Inputs: X (normalized training images)
#  • Targets: X (we want the output to reconstruct the input)
#  • epochs=10: run through the entire dataset 10 times
#  • batch_size=1024: process 1024 samples per gradient update
#  • verbose=2: display one line per epoch with loss/metrics
#  • shuffle=True: shuffle the training data at each epoch for better generalization
#  • validation_split=0.1: use 10% of the training data for validation after each epoch
history = Ae_Conv.fit(
    X, X,
    epochs=10,
    batch_size=1024,
    shuffle=True,
    verbose=2,
    validation_split=0.1
)


Epoch 1/10
53/53 - 121s - 2s/step - accuracy: 0.4941 - loss: 0.4760 - val_accuracy: 0.4953 - val_loss: 0.3624
Epoch 2/10


KeyboardInterrupt: 